In [26]:
import pandas as pd
from app import app, db, Reservation, DeviceStatus, User
from datetime import datetime, timedelta
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

In [27]:
df1 = pd.read_csv('data\Sign up form for Esports room (Antwoorden) - Formulierreacties 1.csv')

<>:1: SyntaxWarning: invalid escape sequence '\S'
<>:1: SyntaxWarning: invalid escape sequence '\S'
C:\Users\pc01\AppData\Local\Temp\ipykernel_26140\2635127739.py:1: SyntaxWarning: invalid escape sequence '\S'
  df1 = pd.read_csv('data\Sign up form for Esports room (Antwoorden) - Formulierreacties 1.csv')


In [28]:
df1

,Tijdstempel,Full name,Student number,Date of visit,Time of visit,What PC/Console will you use?,Free space for comments,Which game are you going to play?,"Which university are you from? (If not from a uni, please let us know from where at the ""other"" section)",E-mailadres,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,19-9-2022 15:41:23,Jian Jin He,193099,3-5-2000,15:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19-9-2022 15:41:27,Thomas Pichardo,223834,19-9-2022,15:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19-9-2022 15:41:31,Samuel Antonio Vieira Vasconcelos,211941,19-9-2001,15:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19-9-2022 15:41:55,Juan José Gutierrez,215517,19-9-2022,15:41:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19-9-2022 15:42:07,Fedor Chursin,220905,19-9-2022,15:42:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,13-9-2024 11:01:38,Patyi Balazs,232737,NaN,NaN,08,NaN,League of Legends,Buas,patyi.balazs14@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2294,13-9-2024 11:53:05,Sem Hooijmans,223121,NaN,NaN,02,NaN,Minecraft,Buas,semhooijmans@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2295,13-9-2024 11:53:38,Rick van den Born,224264,NaN,NaN,01,NaN,NaN,Buas,224264@buas.nl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2296,13-9-2024 14:16:15,Laszlo Oliver,241565,NaN,NaN,05,NaN,NaN,Buas,oli.laszlo.21@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df1 = df1.drop(['Time of visit', 'Date of visit ', 'Free space for comments', 'Which game are you going to play?', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12','Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'], axis=1)

In [30]:
df1

,Tijdstempel,Full name,Student number,What PC/Console will you use?,"Which university are you from? (If not from a uni, please let us know from where at the ""other"" section)",E-mailadres
0,19-9-2022 15:41:23,Jian Jin He,193099,NaN,NaN,NaN
1,19-9-2022 15:41:27,Thomas Pichardo,223834,NaN,NaN,NaN
2,19-9-2022 15:41:31,Samuel Antonio Vieira Vasconcelos,211941,NaN,NaN,NaN
3,19-9-2022 15:41:55,Juan José Gutierrez,215517,NaN,NaN,NaN
4,19-9-2022 15:42:07,Fedor Chursin,220905,NaN,NaN,NaN
...,...,...,...,...,...,...
2293,13-9-2024 11:01:38,Patyi Balazs,232737,08,Buas,patyi.balazs14@gmail.com
2294,13-9-2024 11:53:05,Sem Hooijmans,223121,02,Buas,semhooijmans@gmail.com
2295,13-9-2024 11:53:38,Rick van den Born,224264,01,Buas,224264@buas.nl
2296,13-9-2024 14:16:15,Laszlo Oliver,241565,05,Buas,oli.laszlo.21@gmail.com


In [31]:
# Rename columns to match SQLAlchemy model
df1.rename(columns={
    'Full name': 'name',
    'Which university are you from? (If not from a uni, please let us know from where at the "other" section)': 'school',
    'E-mailadres': 'email',
    'What PC/Console will you use?': 'device',
    'Tijdstempel': 'date'
}, inplace=True)

In [32]:
df1.fillna('N/A')

,date,name,Student number,device,school,email
0,19-9-2022 15:41:23,Jian Jin He,193099,N/A,N/A,N/A
1,19-9-2022 15:41:27,Thomas Pichardo,223834,N/A,N/A,N/A
2,19-9-2022 15:41:31,Samuel Antonio Vieira Vasconcelos,211941,N/A,N/A,N/A
3,19-9-2022 15:41:55,Juan José Gutierrez,215517,N/A,N/A,N/A
4,19-9-2022 15:42:07,Fedor Chursin,220905,N/A,N/A,N/A
...,...,...,...,...,...,...
2293,13-9-2024 11:01:38,Patyi Balazs,232737,08,Buas,patyi.balazs14@gmail.com
2294,13-9-2024 11:53:05,Sem Hooijmans,223121,02,Buas,semhooijmans@gmail.com
2295,13-9-2024 11:53:38,Rick van den Born,224264,01,Buas,224264@buas.nl
2296,13-9-2024 14:16:15,Laszlo Oliver,241565,05,Buas,oli.laszlo.21@gmail.com


In [33]:
# Create SQLAlchemy engine
engine = create_engine('sqlite:///reservations.db')
Session = sessionmaker(bind=engine)
session = Session()

In [34]:
df1

,date,name,Student number,device,school,email
0,19-9-2022 15:41:23,Jian Jin He,193099,NaN,NaN,NaN
1,19-9-2022 15:41:27,Thomas Pichardo,223834,NaN,NaN,NaN
2,19-9-2022 15:41:31,Samuel Antonio Vieira Vasconcelos,211941,NaN,NaN,NaN
3,19-9-2022 15:41:55,Juan José Gutierrez,215517,NaN,NaN,NaN
4,19-9-2022 15:42:07,Fedor Chursin,220905,NaN,NaN,NaN
...,...,...,...,...,...,...
2293,13-9-2024 11:01:38,Patyi Balazs,232737,08,Buas,patyi.balazs14@gmail.com
2294,13-9-2024 11:53:05,Sem Hooijmans,223121,02,Buas,semhooijmans@gmail.com
2295,13-9-2024 11:53:38,Rick van den Born,224264,01,Buas,224264@buas.nl
2296,13-9-2024 14:16:15,Laszlo Oliver,241565,05,Buas,oli.laszlo.21@gmail.com


In [35]:
def map_df_to_model(df):
    reservations = []
    for index, row in df.iterrows():
        # Initialize date variable
        date = None
        
        if isinstance(row['date'], str):
            # Try parsing with different formats
            try:
                # Try with seconds
                date = datetime.strptime(row['date'], '%d-%m-%Y %H:%M:%S')
            except ValueError:
                try:
                    # Try without seconds
                    date = datetime.strptime(row['date'], '%d-%m-%Y %H:%M')
                except ValueError:
                    # Handle if neither format works
                    print(f"Failed to parse date: {row['date']}")
                    continue  # Skip this row if parsing fails
        else:
            # Convert Timestamp to datetime if it's not a string
            date = row['date'].to_pydatetime()

        # Create a Reservation instance
        reservation = Reservation(
            name=row['name'],
            school=row['school'] if pd.notna(row['school']) else 'Unknown',
            email=row['email'] if pd.notna(row['email']) else 'Unknown',
            device=row['device'] if pd.notna(row['device']) else 'Unknown',
            pin_hash='N/A',
            date=date
        )
        reservations.append(reservation)
    
    return reservations


In [36]:
# Insert DataFrame data into the database
reservations = map_df_to_model(df1)

try:
    session.add_all(reservations)
    session.commit()
    print("Data inserted successfully.")
except Exception as e:
    session.rollback()
    print(f"An error occurred: {e}")
finally:
    session.close()

An error occurred: (sqlite3.OperationalError) no such table: reservation
[SQL: INSERT INTO reservation (name, school, email, device, pin_hash, date) VALUES (?, ?, ?, ?, ?, ?) RETURNING id]
[parameters: ('Jian Jin He', 'Unknown', 'Unknown', 'Unknown', 'N/A', '2022-09-19 15:41:23.000000')]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [37]:
# Check current database entries
with app.app_context():
    # Query all reservations
    reservations = Reservation.query.all()
    print("Reservations:", reservations)

    # Query all device statuses
    device_statuses = DeviceStatus.query.all()
    print("Device Statuses:", device_statuses)

    # Query all users
    users = User.query.all()
    print("Users:", users)

    # Query reservations in the last 24 hours
    end_date = datetime.now()
    start_date = end_date - timedelta(days=1)
    reservations_last_24_hours = Reservation.query.filter(
        Reservation.date.between(start_date, end_date)
    ).all()
    print("Reservations in Last 24 Hours:", reservations_last_24_hours)

Reservations: [<Reservation 1 | Name: a | School: BUas | Email: a@email.com | Device: PC1 | Date: 2024-09-13 16:46:44.091518>, <Reservation 2 | Name: b | School: BUas | Email: a@email.com | Device: PC2 | Date: 2024-09-13 16:46:48.159643>]
Device Statuses: [<DeviceStatus 1 | Device: PC1 | Status: Reserved>, <DeviceStatus 2 | Device: PC2 | Status: Reserved>]
Users: []
Reservations in Last 24 Hours: [<Reservation 1 | Name: a | School: BUas | Email: a@email.com | Device: PC1 | Date: 2024-09-13 16:46:44.091518>, <Reservation 2 | Name: b | School: BUas | Email: a@email.com | Device: PC2 | Date: 2024-09-13 16:46:48.159643>]
